## Analyze cross validation results

In [1]:
import collections
import json
import numpy as np
import os
import pandas as pd
import yaml
import pprint
import scipy.stats
import tqdm

pd.set_option("display.max_columns", 100)

In [15]:
# Read yaml files
cross_val_dir = os.path.join(os.getenv("DATA_DIR"),
                             "mica_text_coref/movie_coref/results/coreference/cross_val_Dec06-07")
files_with_empty_results = []
rows = []
for directory in tqdm.tqdm(os.listdir(cross_val_dir), unit="dir"):
    result_file = os.path.join(cross_val_dir, directory, "result.yaml")
    with open(result_file, "r") as f:
        result = yaml.load(f, Loader=yaml.FullLoader)
    file_rows = []
    for document_len, document_len_metric in result["test_metric"].items():
        for overlap_len, overlap_len_metric in document_len_metric.items():
            for strategy, strategy_metric in overlap_len_metric.items():
                row = [result["preprocess"], result["bert_lr"], result["coref_lr"], result["warmup"],
                       document_len, overlap_len, strategy, result["test_movie"]]
                for data_metric in [result["dev_metric"], strategy_metric]:
                    metric_f1s = []
                    for metric in ["muc", "bcub", "ceafe"]:
                        for score in ["precision", "recall", "f1"]:
                            row.append(data_metric["span"][metric][score])
                        metric_f1s.append(data_metric["span"][metric]["f1"])
                    row.append(np.mean(metric_f1s))
                file_rows.append(row)
    if len(file_rows) == 0:
        files_with_empty_results.append(directory)
    rows += file_rows
data = np.array(rows)

hyperparams = ["preprocess", "bert_lr", "model_lr", "warmup", "document_len", "overlap_len", "strategy"]
score_cols = []
for dataset in ["dev", "test"]:
    for metric in ["muc", "bcub", "ceafe"]:
        for score in ["p", "r", "f"]:
            score_cols.append(f"{dataset}_{metric}_{score}")
    score_cols.append(f"{dataset}_conll_f1")
cross_val_df = pd.DataFrame(rows, columns=hyperparams + ["test_movie"] + score_cols)

print(f"shape = {cross_val_df.shape}")
display(cross_val_df)
print(f"directories with empty results: {files_with_empty_results}")
print()

print("Columns:")
for column_name, dtype in zip(cross_val_df.columns[:8], cross_val_df.dtypes[:8]):
    unique_vals = cross_val_df[column_name].unique().tolist()
    print(f"\t{column_name:20s} ({dtype}) : {unique_vals}")

100%|██████████| 719/719 [00:12<00:00, 58.85dir/s]


shape = (2139, 28)


,preprocess,bert_lr,model_lr,warmup,document_len,overlap_len,strategy,test_movie,dev_muc_p,dev_muc_r,dev_muc_f,dev_bcub_p,dev_bcub_r,dev_bcub_f,dev_ceafe_p,dev_ceafe_r,dev_ceafe_f,dev_conll_f1,test_muc_p,test_muc_r,test_muc_f,test_bcub_p,test_bcub_r,test_bcub_f,test_ceafe_p,test_ceafe_r,test_ceafe_f,test_conll_f1
0,addsays,0.00005,0.0002,0.0,20480,512,avg,zootopia,89.98,90.44,90.21,72.01,78.08,74.92,61.67,45.38,52.29,72.473333,93.20,91.27,92.23,79.95,64.08,71.14,45.46,33.39,38.50,67.290000
1,addsays,0.00005,0.0002,0.0,20480,1024,avg,zootopia,89.98,90.44,90.21,72.01,78.08,74.92,61.67,45.38,52.29,72.473333,93.02,91.30,92.15,79.56,64.19,71.06,44.44,34.21,38.66,67.290000
2,addsays,0.00005,0.0002,0.0,20480,2048,avg,zootopia,89.98,90.44,90.21,72.01,78.08,74.92,61.67,45.38,52.29,72.473333,93.15,91.27,92.20,76.97,63.21,69.42,47.42,35.67,40.71,67.443333
3,regular,0.00002,0.0001,0.5,10240,512,avg,john_wick,89.98,91.18,90.58,68.54,73.87,71.11,53.64,45.04,48.96,70.216667,94.03,92.93,93.48,82.81,74.27,78.31,37.11,70.94,48.73,73.506667
4,regular,0.00002,0.0001,0.5,10240,1024,avg,john_wick,89.98,91.18,90.58,68.54,73.87,71.11,53.64,45.04,48.96,70.216667,93.88,92.81,93.34,85.61,77.09,81.13,43.20,74.96,54.81,76.426667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2134,regular,0.00002,0.0001,-1.0,10240,1024,avg,prestige,90.50,90.74,90.62,80.18,76.47,78.28,61.26,46.81,53.07,73.990000,93.30,93.41,93.36,78.90,73.68,76.20,29.78,66.57,41.15,70.236667
2135,regular,0.00002,0.0001,-1.0,10240,2048,avg,prestige,90.50,90.74,90.62,80.18,76.47,78.28,61.26,46.81,53.07,73.990000,93.40,93.43,93.42,79.28,63.75,70.67,30.36,66.97,41.78,68.623333
2136,regular,0.00001,0.0002,-1.0,20480,512,avg,avengers_endgame,88.92,91.52,90.20,62.11,77.18,68.83,52.19,41.85,46.45,68.493333,92.26,92.91,92.58,48.44,58.08,52.82,29.97,42.21,35.05,60.150000
2137,regular,0.00001,0.0002,-1.0,20480,1024,avg,avengers_endgame,88.92,91.52,90.20,62.11,77.18,68.83,52.19,41.85,46.45,68.493333,92.29,92.83,92.56,44.18,59.34,50.65,29.64,37.58,33.14,58.783333


directories with empty results: ['Dec07_02:10:59AM_avengers_endgame', 'Dec07_03:11:04AM_john_wick', 'Dec07_02:11:39AM_avengers_endgame', 'Dec06_09:09:18AM_zootopia', 'Dec07_04:14:02AM_dead_poets_society', 'Dec06_10:32:19AM_zootopia']

Columns:
	preprocess           (object) : ['addsays', 'regular']
	bert_lr              (float64) : [5e-05, 2e-05, 1e-05]
	model_lr             (float64) : [0.0002, 0.0001]
	warmup               (float64) : [0.0, 0.5, 0.25, 1.0, -1.0]
	document_len         (int64) : [20480, 10240]
	overlap_len          (int64) : [512, 1024, 2048]
	strategy             (object) : ['avg']
	test_movie           (object) : ['zootopia', 'john_wick', 'quiet_place', 'avengers_endgame', 'dead_poets_society', 'prestige']


In [17]:
# Aggregate (macro) coreference performance scores for each movie
index_rows = []
data_rows = []
hyperparams = ["preprocess", "bert_lr", "model_lr", "warmup", "document_len", "overlap_len", "strategy"]
for hparam_values, df in cross_val_df.groupby(hyperparams):
    if len(df) == 6:
        row = df.iloc[:,len(hparam_values) + 1:].agg(["mean", "std"]).values.T.flatten().tolist()
        index_rows.append(list(hparam_values))
        data_rows.append(row)

index = pd.MultiIndex.from_product([["dev"], ["muc", "bcub", "ceafe"], ["p", "r", "f"], ["mean", "std"]],
                                   sortorder=None)
index = index.union(pd.MultiIndex.from_product([["dev"], ["conll"], ["f"], ["mean", "std"]], sortorder=None),
                    sort=False)
index = index.union(pd.MultiIndex.from_product([["test"], ["muc", "bcub", "ceafe"], ["p", "r", "f"], ["mean", "std"]],
                                               sortorder=None), sort=False)
index = index.union(pd.MultiIndex.from_product([["test"], ["conll"], ["f"], ["mean", "std"]], sortorder=None),
                    sort=False)
index.names = ["dataset", "metric", "score", "func"]

cross_val_index = pd.MultiIndex.from_frame(pd.DataFrame(index_rows, columns=hyperparams))
cross_val_agg_df = pd.DataFrame(data_rows, columns=index, index=cross_val_index)

display(cross_val_agg_df)
print(f"shape = {cross_val_agg_df.shape}")

dataset                                                                     dev  \
metric                                                                      muc   
score                                                                         p   
func                                                                       mean   
preprocess bert_lr model_lr warmup document_len overlap_len strategy              
addsays    0.00001 0.0001   -1.0   10240        512         avg       89.785000   
                                                1024        avg       89.785000   
                                                2048        avg       89.785000   
                                   20480        512         avg       89.785000   
                                                1024        avg       89.785000   
...                                                                         ...   
regular    0.00005 0.0002    1.0   10240        1024        avg       90.156667   
                                                2048        avg       90.156667   
                                   20480        512         avg       90.156667   
                                                1024        avg       90.156667   
                                                2048        avg       90.156667   

dataset                                                                         \
metric                                                                           
score                                                                            
func                                                                       std   
preprocess bert_lr model_lr warmup document_len overlap_len strategy             
addsays    0.00001 0.0001   -1.0   10240        512         avg       0.557127   
                                                1024        avg       0.557127   
                                                2048        avg       0.557127   
                                   20480        512         avg       0.557127   
                                                1024        avg       0.557127   
...                                                                        ...   
regular    0.00005 0.0002    1.0   10240        1024        avg       2.260864   
                                                2048        avg       2.260864   
                                   20480        512         avg       2.260864   
                                                1024        avg       2.260864   
                                                2048        avg       2.260864   

dataset                                                                          \
metric                                                                            
score                                                                         r   
func                                                                       mean   
preprocess bert_lr model_lr warmup document_len overlap_len strategy              
addsays    0.00001 0.0001   -1.0   10240        512         avg       91.993333   
                                                1024        avg       91.993333   
                                                2048        avg       91.993333   
                                   20480        512         avg       91.993333   
                                                1024        avg       91.993333   
...                                                                         ...   
regular    0.00005 0.0002    1.0   10240        1024        avg       86.000000   
                                                2048        avg       86.000000   
                                   20480        512         avg       86.000000   
                                                1024        avg       86.000000   
                                                2048        avg       86.000000   

dataset                      

shape = (339, 40)


In [29]:
# Find hyperparameter settings for which results are empty
index_frame = cross_val_agg_df.index.to_frame(index=False)
allpermute_index_frame = pd.MultiIndex.from_product([["addsays", "regular"], [1e-5, 2e-5, 5e-5], [1e-4, 2e-4],
                                                     [-1, 0, 0.25, 0.5, 1], [10240, 20480], [512, 1024, 2048],
                                                     ["avg"]]).to_frame(index=False)
allpermute_index_frame.columns = hyperparams
indicator = allpermute_index_frame.merge(index_frame, how="left", indicator=True)["_merge"]
print("hyperparam settings for which results are empty:")
display(allpermute_index_frame[indicator == "left_only"].sort_values(["preprocess", "bert_lr", "model_lr", "warmup"]))

hyperparam settings for which results are empty:


,preprocess,bert_lr,model_lr,warmup,document_len,overlap_len,strategy
132,addsays,0.00005,0.0001,0.25,10240,512,avg
133,addsays,0.00005,0.0001,0.25,10240,1024,avg
134,addsays,0.00005,0.0001,0.25,10240,2048,avg
144,addsays,0.00005,0.0001,1.00,10240,512,avg
145,addsays,0.00005,0.0001,1.00,10240,1024,avg
146,addsays,0.00005,0.0001,1.00,10240,2048,avg
147,addsays,0.00005,0.0001,1.00,20480,512,avg
148,addsays,0.00005,0.0001,1.00,20480,1024,avg
149,addsays,0.00005,0.0001,1.00,20480,2048,avg
156,addsays,0.00005,0.0002,0.00,10240,512,avg


In [38]:
cross_val_agg_df.loc[cross_val_agg_df["test"]["conll"]["f"]["mean"] == cross_val_agg_df["test"]["conll"]["f"]["mean"].max(), ("test", slice(None), "f")]

dataset                                                                            test  \
metric                                                                              muc   
score                                                                                 f   
func                                                                               mean   
preprocess bert_lr model_lr warmup document_len overlap_len strategy                      
regular    2e-05   0.0001   -1.0   20480        2048        avg       93.62166666666667   

dataset                                                                                  \
metric                                                                                    
score                                                                                     
func                                                                                std   
preprocess bert_lr model_lr warmup document_len overlap_len strategy                      
regular    2e-05   0.0001   -1.0   20480        2048        avg       1.223460937940671   

dataset                                                                     \
metric                                                                bcub   
score                                                                    f   
func                                                                  mean   
preprocess bert_lr model_lr warmup document_len overlap_len strategy         
regular    2e-05   0.0001   -1.0   20480        2048        avg       73.2   

dataset                                                                                   \
metric                                                                                     
score                                                                                      
func                                                                                 std   
preprocess bert_lr model_lr warmup document_len overlap_len strategy                       
regular    2e-05   0.0001   -1.0   20480        2048        avg       11.976625568164014   

dataset                                                                                   \
metric                                                                             ceafe   
score                                                                                  f   
func                                                                                mean   
preprocess bert_lr model_lr warmup document_len overlap_len strategy                       
regular    2e-05   0.0001   -1.0   20480        2048        avg       46.708333333333336   

dataset                                                                                  \
metric                                                                                    
score                                                                                     
func                                                                                std   
preprocess bert_lr model_lr warmup document_len overlap_len strategy                      
regular    2e-05   0.0001   -1.0   20480        2048        avg       8.389394296769384   

dataset                                                                                  \
metric                                                                            conll   
score                                                                                 f   
func                                                                               mean   
preprocess bert_lr model_lr warmup document_len overlap_len strategy                      
regular    2e-05   0.0001   -1.0   20480        2048        avg       71.17666666666666   

dataset                                                                                 
metric                                                                                  
score                                                      

In [35]:
cross_val_agg_df[cross_val_agg_df["dev"]["conll"]["f"]["mean"] == cross_val_agg_df["dev"]["conll"]["f"]["mean"].max()]

dataset                                                                             dev  \
metric                                                                              muc   
score                                                                                 p   
func                                                                               mean   
preprocess bert_lr model_lr warmup document_len overlap_len strategy                      
addsays    5e-05   0.0001   0.0    10240        512         avg       88.82833333333332   
                                                1024        avg       88.82833333333332   
                                                2048        avg       88.82833333333332   
                                   20480        512         avg       88.82833333333333   
                                                1024        avg       88.82833333333333   
                                                2048        avg       88.82833333333333   

dataset                                                                                   \
metric                                                                                     
score                                                                                      
func                                                                                 std   
preprocess bert_lr model_lr warmup document_len overlap_len strategy                       
addsays    5e-05   0.0001   0.0    10240        512         avg       0.9350169338929974   
                                                1024        avg       0.9350169338929974   
                                                2048        avg       0.9350169338929974   
                                   20480        512         avg       0.9350169338929974   
                                                1024        avg       0.9350169338929974   
                                                2048        avg       0.9350169338929974   

dataset                                                                                  \
metric                                                                                    
score                                                                                 r   
func                                                                               mean   
preprocess bert_lr model_lr warmup document_len overlap_len strategy                      
addsays    5e-05   0.0001   0.0    10240        512         avg       90.61166666666666   
                                                1024        avg       90.61166666666666   
                                                2048        avg       90.61166666666666   
                                   20480        512         avg       90.61166666666666   
                                                1024        avg       90.61166666666666   
                                                2048        avg       90.61166666666666   

dataset                                                                                   \
metric                                                                                     
score                                                                                      
func                                                                                 std   
preprocess bert_lr model_lr warmup document_len overlap_len strategy                       
addsays    5e-05   0.0001   0.0    10240        512         avg       1.2286320306205043   
                                                1024        avg       1.2286320306205043   
                                                2048        avg       1.2286320306205043   
                                   20480        512         avg       1.2286320306205043   
                                                1024        avg       1.2286320306205043   
                                                2048        avg